### 2.5.5. Default Document queries(修改默认文档查询方法)

By default, the objects `objects` attribute on a document returns a `QuerySet` that doesn’t filter the collection – it returns all objects. This may be changed by defining a method on a document that modifies a queryset. The method should accept two arguments – `doc_cls` and `queryset`. The first argument is the `Document` class that the method is defined on (in this sense, the method is more like a `classmethod()` than a regular method), and the second argument is the initial queryset. The method needs to be decorated with `queryset_manager()` in order for it to be recognised.

可以为文档类定义默认查询方法。可以修改原来的 `objects`方法，也可以定义单独的 查询方法。

```python
class BlogPost(Document):
    title = StringField()
    date = DateTimeField()

    @queryset_manager
    def objects(doc_cls, queryset):
        # This may actually also be done by defining a default ordering for
        # the document, but this illustrates the use of manager methods
        return queryset.order_by('-date')
```

You don’t need to call your method `objects` – you may define as many custom manager methods as you like:

```python
class BlogPost(Document):
    title = StringField()
    published = BooleanField()

    @queryset_manager
    def live_posts(doc_cls, queryset):
        return queryset.filter(published=True)

BlogPost(title='test1', published=False).save()
BlogPost(title='test2', published=True).save()
assert len(BlogPost.objects) == 2
assert len(BlogPost.live_posts()) == 1
```

### 2.5.6. Custom QuerySets(另一种定制化查询的方法)

Should you want to add custom methods for interacting with or filtering documents, extending the `QuerySet` class may be the way to go. To use a custom `QuerySet` class on a document, set `queryset_class` to the custom class in a Document’s `meta` dictionary:

```python
class AwesomerQuerySet(QuerySet):

    def get_awesome(self):
        return self.filter(awesome=True)

class Page(Document):
    meta = {'queryset_class': AwesomerQuerySet}

# To call:
Page.objects.get_awesome()
```